## Muito semelhante ao Train:
# Importante: se alguma coisa do train for modificada, modifique aqui também!!!!

### Importar bibliotecas necessárias

In [13]:
import numpy
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## Ler Base de dados
## CUIDADO COM O SEPARADOR DO CSV (trocar ; por ,)

In [14]:
data = pd.read_csv('our_clean.csv')

In [15]:
data['seller_id'] = data['seller_id'].apply(str)
data['category_id'] = data['category_id'].apply(str)

## Separar label da feature

In [16]:
x_data = data

## Criar as feature columns categoricas

In [17]:
ad_type_description = tf.feature_column.categorical_column_with_hash_bucket('ad_type_description',hash_bucket_size=3)
shipping_id = tf.feature_column.categorical_column_with_identity('shipping_id', num_buckets=8)
category_id = tf.feature_column.categorical_column_with_hash_bucket('category_id', hash_bucket_size=300)

# NÃO PASSE MAIS DE 10 PRODUTOS DE UMA VEZ!!!!!!!!!!!!!!!

In [18]:
seller_id = tf.feature_column.categorical_column_with_hash_bucket('seller_id', hash_bucket_size=10000)

## Passar as feature columns categoricas para embedded

In [19]:
ad_type_description = tf.feature_column.embedding_column(ad_type_description, dimension=3)
shipping_id = tf.feature_column.embedding_column(shipping_id, dimension=8)
category_id = tf.feature_column.embedding_column(category_id, dimension=300)
seller_id = tf.feature_column.embedding_column(seller_id, dimension=10000)

## Criar feature columns numéricas

In [20]:
price = tf.feature_column.numeric_column('price')

sold_quantity_0 = tf.feature_column.numeric_column('sold_qty_0')
sold_quantity_1 = tf.feature_column.numeric_column('sold_qty_1')
sold_quantity_2 = tf.feature_column.numeric_column('sold_qty_2')
sold_quantity_3 = tf.feature_column.numeric_column('sold_qty_3')
sold_quantity_4 = tf.feature_column.numeric_column('sold_qty_4')
sold_quantity_5 = tf.feature_column.numeric_column('sold_qty_5')
sold_last_week = tf.feature_column.numeric_column('sold_last_week')

position_0 = tf.feature_column.numeric_column('pos_0')
position_1 = tf.feature_column.numeric_column('pos_1')
position_2 = tf.feature_column.numeric_column('pos_2')
position_3 = tf.feature_column.numeric_column('pos_3')
position_4 = tf.feature_column.numeric_column('pos_4')
position_5 = tf.feature_column.numeric_column('pos_5')

## Completar feature columns

In [21]:
feat_cols = [ad_type_description,shipping_id,category_id,seller_id,price,sold_quantity_0,
             sold_quantity_1,sold_quantity_2,sold_quantity_3,sold_quantity_4,sold_quantity_5,
             position_0,position_1,position_2,position_3,position_4,position_5,sold_last_week]

## Criação do modelo (O modelo salva automaticamente após ser treinado)
# CUIDADO COM O NUMERO DE NEURONIOS, TEM QUE SER IGUAL AO TREINAMENTO!!!!!

In [22]:
model = tf.estimator.DNNClassifier([18],feature_columns=feat_cols,n_classes=2,model_dir="./model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_tf_random_seed': 1, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_session_config': None, '_model_dir': './model', '_keep_checkpoint_max': 5, '_save_summary_steps': 100}


## Input da Previsão

In [23]:
pred_func=tf.estimator.inputs.pandas_input_fn(x=x_data,batch_size=len(x_data),shuffle=False)

## Previsão

In [24]:
prediction = model.predict(pred_func)
prediction = list(prediction)
final_preds = [pred['class_ids'][0] for pred in prediction]
print(final_preds)
print(len(final_preds))

INFO:tensorflow:Restoring parameters from ./model\model.ckpt-36000
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
24


### Salva as previsões para serem unidas à base de dados

In [13]:
numpy.savetxt("prediction.csv", final_preds, delimiter=",")